# COMP 3610: Big Data Analytics - Assignment 1
## Data Pipeline & Visualization Dashboard

**Student ID:** 816034871

**Dataset:** NYC Yellow Taxi Trip Data (January 2024)

This notebook demonstrates the end-to-end data pipeline including:
- Part 1: Data Ingestion & Storage
- Part 2: Data Transformation & Analysis
- Part 3: Visualization Prototypes

---

## Setup and Imports

In [ ]:
# Install required packages (run once)
# !pip install pandas pyarrow duckdb requests plotly streamlit

In [ ]:
# Core imports
import os
import requests
from pathlib import Path
from datetime import datetime

# Data processing
import pandas as pd
import duckdb

# Visualization
import plotly.express as px
import plotly.graph_objects as go

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("All imports successful!")
print(f"Pandas version: {pd.__version__}")
print(f"DuckDB version: {duckdb.__version__}")

---
# Part 1: Data Ingestion & Storage (20 marks)
---

## 1.1 Programmatic Download (5 marks)

Download the NYC Yellow Taxi Trip data and Taxi Zone Lookup table programmatically using Python's `requests` library.

In [ ]:
# Define data URLs
TAXI_DATA_URL = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
ZONE_LOOKUP_URL = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"

# Define data directory structure
DATA_DIR = Path("data")
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"

# Create directories if they don't exist
RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print(f"Data directories created:")
print(f"  - Raw data: {RAW_DIR.absolute()}")
print(f"  - Processed data: {PROCESSED_DIR.absolute()}")

In [ ]:
def download_file(url: str, destination: Path, chunk_size: int = 8192) -> bool:
    """
    Download a file from a URL with progress tracking.
    
    Args:
        url: The URL to download from
        destination: The local file path to save to
        chunk_size: Size of chunks to download at a time
    
    Returns:
        bool: True if download successful, raises exception otherwise
    """
    print(f"Downloading: {url}")
    print(f"Destination: {destination}")
    
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise exception for HTTP errors
        
        # Get file size if available
        total_size = int(response.headers.get('content-length', 0))
        
        downloaded = 0
        with open(destination, 'wb') as f:
            for chunk in response.iter_content(chunk_size=chunk_size):
                if chunk:
                    f.write(chunk)
                    downloaded += len(chunk)
                    if total_size > 0:
                        progress = (downloaded / total_size) * 100
                        print(f"\rProgress: {progress:.1f}% ({downloaded / 1024 / 1024:.1f} MB)", end="")
        
        print(f"\nDownload complete! File size: {destination.stat().st_size / 1024 / 1024:.2f} MB")
        return True
        
    except requests.exceptions.RequestException as e:
        raise Exception(f"Failed to download {url}: {str(e)}")

In [ ]:
# Define file paths
taxi_data_path = RAW_DIR / "yellow_tripdata_2024-01.parquet"
zone_lookup_path = RAW_DIR / "taxi_zone_lookup.csv"

# Download taxi trip data (if not already downloaded)
if not taxi_data_path.exists():
    download_file(TAXI_DATA_URL, taxi_data_path)
else:
    print(f"Taxi data already exists: {taxi_data_path}")
    print(f"File size: {taxi_data_path.stat().st_size / 1024 / 1024:.2f} MB")

In [ ]:
# Download zone lookup table (if not already downloaded)
if not zone_lookup_path.exists():
    download_file(ZONE_LOOKUP_URL, zone_lookup_path)
else:
    print(f"Zone lookup already exists: {zone_lookup_path}")
    print(f"File size: {zone_lookup_path.stat().st_size / 1024:.2f} KB")

## 1.2 Data Validation (10 marks)

Validate the downloaded data by:
- Verifying expected columns exist
- Checking datetime types
- Reporting row counts
- Raising exceptions on validation failure

In [ ]:
# Define expected columns per assignment specification (Dataset Schema)
EXPECTED_COLUMNS = [
    'tpep_pickup_datetime',
    'tpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'fare_amount',
    'tip_amount',
    'total_amount',
    'payment_type'
]

DATETIME_COLUMNS = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']

In [ ]:
def validate_taxi_data(df: pd.DataFrame) -> dict:
    """
    Validate the taxi trip data and return validation results.
    
    Args:
        df: The taxi trip DataFrame to validate
    
    Returns:
        dict: Validation results including status and details
    
    Raises:
        ValueError: If validation fails
    """
    results = {
        'row_count': len(df),
        'column_count': len(df.columns),
        'columns_present': list(df.columns),
        'missing_columns': [],
        'datetime_valid': True,
        'validation_passed': True,
        'errors': []
    }
    
    # Check for expected columns
    for col in EXPECTED_COLUMNS:
        if col not in df.columns:
            results['missing_columns'].append(col)
    
    if results['missing_columns']:
        results['validation_passed'] = False
        results['errors'].append(f"Missing columns: {results['missing_columns']}")
    
    # Check datetime columns
    for col in DATETIME_COLUMNS:
        if col in df.columns:
            if not pd.api.types.is_datetime64_any_dtype(df[col]):
                results['datetime_valid'] = False
                results['errors'].append(f"Column {col} is not datetime type")
    
    # Check row count is reasonable (should be > 0)
    if results['row_count'] == 0:
        results['validation_passed'] = False
        results['errors'].append("DataFrame is empty")
    
    if not results['validation_passed']:
        raise ValueError(f"Data validation failed: {results['errors']}")
    
    return results

In [ ]:
# Load and validate the taxi trip data
print("Loading taxi trip data...")
df_taxi = pd.read_parquet(taxi_data_path)

print("\nValidating data...")
validation_results = validate_taxi_data(df_taxi)

print("\n" + "="*50)
print("VALIDATION RESULTS")
print("="*50)
print(f"✓ Total rows: {validation_results['row_count']:,}")
print(f"✓ Total columns: {validation_results['column_count']}")
print(f"✓ Datetime columns valid: {validation_results['datetime_valid']}")
print(f"✓ Validation passed: {validation_results['validation_passed']}")
print("="*50)

In [ ]:
# Load zone lookup data
print("Loading zone lookup data...")
df_zones = pd.read_csv(zone_lookup_path)

print(f"\nZone Lookup Table:")
print(f"  Rows: {len(df_zones):,}")
print(f"  Columns: {list(df_zones.columns)}")
df_zones.head()

In [ ]:
# Display data info
print("\nTaxi Data Info:")
print(df_taxi.info())
print("\n" + "="*50)
print("\nFirst 5 rows:")
df_taxi.head()

## 1.3 File Organization (5 marks)

Data files are saved to `data/raw/` directory. The `.gitignore` file excludes the data directory from version control to avoid committing large data files.

In [ ]:
# Display file organization
print("Project File Organization:")
print("")
print("assignment1/")
print("├── data/")
print("│   ├── raw/")
print("│   │   ├── yellow_tripdata_2024-01.parquet")
print("│   │   └── taxi_zone_lookup.csv")
print("│   └── processed/")
print("├── assignment1.ipynb")
print("├── app.py")
print("├── requirements.txt")
print("├── README.md")
print("└── .gitignore  (excludes data/ directory)")

---
# Part 2: Data Transformation & Analysis (30 marks)
---

## 2.1 Data Cleaning (10 marks)

Clean the data by:
- Removing null values
- Removing invalid trips (zero/negative distance, fare, or duration)
- Documenting all removals

In [ ]:
def clean_taxi_data(df: pd.DataFrame) -> tuple[pd.DataFrame, dict]:
    """
    Clean the taxi trip data and document removals.
    
    Args:
        df: Raw taxi trip DataFrame
    
    Returns:
        tuple: (cleaned DataFrame, cleaning statistics dict)
    
    Raises:
        ValueError: If cleaning removes all rows
    """
    original_count = len(df)
    cleaning_stats = {
        'original_rows': original_count,
        'removals': {}
    }
    
    # Make a copy to avoid modifying original
    df_clean = df.copy()
    
    # 1. Remove rows with null values in critical columns
    critical_columns = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 
                        'trip_distance', 'fare_amount', 'total_amount',
                        'PULocationID', 'DOLocationID']
    
    before = len(df_clean)
    df_clean = df_clean.dropna(subset=critical_columns)
    cleaning_stats['removals']['null_values'] = before - len(df_clean)
    
    # 2. Remove trips with zero or negative distance
    before = len(df_clean)
    df_clean = df_clean[df_clean['trip_distance'] > 0]
    cleaning_stats['removals']['invalid_distance'] = before - len(df_clean)
    
    # 3. Remove trips with zero or negative fare
    before = len(df_clean)
    df_clean = df_clean[df_clean['fare_amount'] > 0]
    cleaning_stats['removals']['invalid_fare'] = before - len(df_clean)
    
    # 4. Remove trips with fares exceeding $500
    before = len(df_clean)
    df_clean = df_clean[df_clean['fare_amount'] <= 500]
    cleaning_stats['removals']['fare_over_500'] = before - len(df_clean)
    
    # 5. Remove trips with invalid total amount
    before = len(df_clean)
    df_clean = df_clean[df_clean['total_amount'] > 0]
    cleaning_stats['removals']['invalid_total'] = before - len(df_clean)
    
    # 6. Remove trips where dropoff is before pickup
    before = len(df_clean)
    df_clean = df_clean[df_clean['tpep_dropoff_datetime'] > df_clean['tpep_pickup_datetime']]
    cleaning_stats['removals']['dropoff_before_pickup'] = before - len(df_clean)
    
    # 7. Calculate duration and remove trips with invalid duration
    df_clean['duration_seconds'] = (df_clean['tpep_dropoff_datetime'] - 
                                     df_clean['tpep_pickup_datetime']).dt.total_seconds()
    
    before = len(df_clean)
    # Remove trips shorter than 1 minute or longer than 24 hours
    df_clean = df_clean[(df_clean['duration_seconds'] >= 60) & 
                        (df_clean['duration_seconds'] <= 86400)]
    cleaning_stats['removals']['invalid_duration'] = before - len(df_clean)
    
    # 8. Remove trips with unrealistic speeds (> 100 mph average)
    df_clean['avg_speed_mph'] = df_clean['trip_distance'] / (df_clean['duration_seconds'] / 3600)
    before = len(df_clean)
    df_clean = df_clean[df_clean['avg_speed_mph'] <= 100]
    cleaning_stats['removals']['unrealistic_speed'] = before - len(df_clean)
    
    # Drop temporary columns
    df_clean = df_clean.drop(columns=['duration_seconds', 'avg_speed_mph'])
    
    # Calculate final statistics
    cleaning_stats['final_rows'] = len(df_clean)
    cleaning_stats['total_removed'] = original_count - len(df_clean)
    cleaning_stats['removal_percentage'] = (cleaning_stats['total_removed'] / original_count) * 100
    
    return df_clean, cleaning_stats

In [ ]:
# Clean the data
print("Cleaning taxi trip data...")
df_clean, cleaning_stats = clean_taxi_data(df_taxi)

print("\n" + "="*50)
print("DATA CLEANING REPORT")
print("="*50)
print(f"Original rows: {cleaning_stats['original_rows']:,}")
print(f"\nRows removed by reason:")
for reason, count in cleaning_stats['removals'].items():
    print(f"  - {reason}: {count:,}")
print(f"\nTotal rows removed: {cleaning_stats['total_removed']:,} ({cleaning_stats['removal_percentage']:.2f}%)")
print(f"Final rows: {cleaning_stats['final_rows']:,}")
print("="*50)

## 2.2 Feature Engineering (10 marks)

Create exactly 4 new derived columns:
1. `trip_duration_minutes` - Calculated from pickup and dropoff timestamps
2. `trip_speed_mph` - Distance divided by duration (handle division by zero)
3. `pickup_hour` - Hour of day (0-23) extracted from pickup timestamp
4. `pickup_day_of_week` - Day name (Monday-Sunday) extracted from pickup timestamp

In [ ]:
def add_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add engineered features to the taxi trip data.
    
    Args:
        df: Cleaned taxi trip DataFrame
    
    Returns:
        DataFrame with new feature columns
    """
    df = df.copy()
    
    # 1. Trip duration in minutes
    df['trip_duration_minutes'] = (
        (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime'])
        .dt.total_seconds() / 60
    ).round(2)
    
    # 2. Average trip speed in MPH (handle division by zero)
    df['trip_speed_mph'] = (
        df['trip_distance'] / (df['trip_duration_minutes'] / 60)
    ).round(2)
    
    # 3. Pickup hour (0-23)
    df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour
    
    # 4. Pickup day of week (day name: Monday-Sunday)
    df['pickup_day_of_week'] = df['tpep_pickup_datetime'].dt.day_name()
    
    return df

In [ ]:
# Add features to the cleaned data
print("Adding engineered features...")
df_features = add_features(df_clean)

print("\nNew columns added:")
new_cols = ['trip_duration_minutes', 'trip_speed_mph', 'pickup_hour', 'pickup_day_of_week']
for col in new_cols:
    print(f"  - {col}")

print("\nSample of new features:")
df_features[['tpep_pickup_datetime', 'trip_distance'] + new_cols].head(10)

In [ ]:
# Summary statistics for new features
print("\nFeature Statistics:")
print("="*50)
print(f"\nTrip Duration (minutes):")
print(f"  Mean: {df_features['trip_duration_minutes'].mean():.2f}")
print(f"  Median: {df_features['trip_duration_minutes'].median():.2f}")
print(f"  Min: {df_features['trip_duration_minutes'].min():.2f}")
print(f"  Max: {df_features['trip_duration_minutes'].max():.2f}")

print(f"\nTrip Speed (MPH):")
print(f"  Mean: {df_features['trip_speed_mph'].mean():.2f}")
print(f"  Median: {df_features['trip_speed_mph'].median():.2f}")

print(f"\nTrips by Day of Week:")
print(df_features['pickup_day_of_week'].value_counts().sort_index())

## 2.3 SQL Analysis with DuckDB (10 marks)

Perform the following SQL queries using DuckDB:
1. Top 10 busiest pickup zones
2. Average fare by hour of day
3. Payment type distribution (percentages)
4. Average tip percentage by day of week (credit card only)
5. Top 5 pickup-dropoff zone pairs

In [ ]:
# Create DuckDB connection and register DataFrames
con = duckdb.connect()
con.register('trips', df_features)
con.register('zones', df_zones)

print("DuckDB connection established and DataFrames registered.")

In [ ]:
# Query 1: Top 10 busiest pickup zones
print("\n" + "="*60)
print("QUERY 1: Top 10 Busiest Pickup Zones")
print("="*60)

query1 = """
SELECT 
    z.Zone as pickup_zone,
    z.Borough as borough,
    COUNT(*) as trip_count,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM trips), 2) as percentage
FROM trips t
JOIN zones z ON t.PULocationID = z.LocationID
GROUP BY z.Zone, z.Borough
ORDER BY trip_count DESC
LIMIT 10
"""

df_query1 = con.execute(query1).fetchdf()
print(df_query1.to_string(index=False))

In [ ]:
# Query 2: Average fare by hour of day
print("\n" + "="*60)
print("QUERY 2: Average Fare by Hour of Day")
print("="*60)

query2 = """
SELECT 
    pickup_hour,
    ROUND(AVG(fare_amount), 2) as avg_fare,
    ROUND(AVG(total_amount), 2) as avg_total,
    COUNT(*) as trip_count
FROM trips
GROUP BY pickup_hour
ORDER BY pickup_hour
"""

df_query2 = con.execute(query2).fetchdf()
print(df_query2.to_string(index=False))

In [ ]:
# Query 3: Payment type distribution (percentages)
print("\n" + "="*60)
print("QUERY 3: Payment Type Distribution")
print("="*60)

query3 = """
SELECT 
    CASE payment_type
        WHEN 1 THEN 'Credit Card'
        WHEN 2 THEN 'Cash'
        WHEN 3 THEN 'No Charge'
        WHEN 4 THEN 'Dispute'
        WHEN 5 THEN 'Unknown'
        WHEN 6 THEN 'Voided Trip'
        ELSE 'Other'
    END as payment_method,
    COUNT(*) as trip_count,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM trips), 2) as percentage
FROM trips
GROUP BY payment_type
ORDER BY trip_count DESC
"""

df_query3 = con.execute(query3).fetchdf()
print(df_query3.to_string(index=False))

In [ ]:
# Query 4: Average tip percentage by day of week (credit card only)
print("\n" + "="*60)
print("QUERY 4: Average Tip Percentage by Day (Credit Card Only)")
print("="*60)

query4 = """
SELECT 
    pickup_day_of_week as day_of_week,
    COUNT(*) as trip_count,
    ROUND(AVG(tip_amount), 2) as avg_tip,
    ROUND(AVG(tip_amount / NULLIF(fare_amount, 0) * 100), 2) as avg_tip_percentage
FROM trips
WHERE payment_type = 1  -- Credit card only
    AND fare_amount > 0
GROUP BY pickup_day_of_week
ORDER BY CASE pickup_day_of_week
    WHEN 'Monday' THEN 1
    WHEN 'Tuesday' THEN 2
    WHEN 'Wednesday' THEN 3
    WHEN 'Thursday' THEN 4
    WHEN 'Friday' THEN 5
    WHEN 'Saturday' THEN 6
    WHEN 'Sunday' THEN 7
END
"""

df_query4 = con.execute(query4).fetchdf()
print(df_query4.to_string(index=False))

In [ ]:
# Query 5: Top 5 pickup-dropoff zone pairs
print("\n" + "="*60)
print("QUERY 5: Top 5 Pickup-Dropoff Zone Pairs")
print("="*60)

query5 = """
SELECT 
    pz.Zone as pickup_zone,
    dz.Zone as dropoff_zone,
    COUNT(*) as trip_count,
    ROUND(AVG(t.trip_distance), 2) as avg_distance,
    ROUND(AVG(t.total_amount), 2) as avg_total
FROM trips t
JOIN zones pz ON t.PULocationID = pz.LocationID
JOIN zones dz ON t.DOLocationID = dz.LocationID
GROUP BY pz.Zone, dz.Zone
ORDER BY trip_count DESC
LIMIT 5
"""

df_query5 = con.execute(query5).fetchdf()
print(df_query5.to_string(index=False))

---
# Part 3: Visualization Prototypes (40 marks)
---

Prototype visualizations for the Streamlit dashboard using Plotly.

## 3.1 Key Metrics

In [ ]:
# Calculate key metrics
metrics = {
    'Total Trips': f"{len(df_features):,}",
    'Average Fare': f"${df_features['fare_amount'].mean():.2f}",
    'Total Revenue': f"${df_features['total_amount'].sum():,.2f}",
    'Average Distance': f"{df_features['trip_distance'].mean():.2f} miles",
    'Average Duration': f"{df_features['trip_duration_minutes'].mean():.1f} mins"
}

print("\n" + "="*60)
print("KEY METRICS")
print("="*60)
for metric, value in metrics.items():
    print(f"{metric}: {value}")

## 3.2 Visualization 1: Top 10 Pickup Zones (Bar Chart)

In [ ]:
# Bar chart: Top 10 pickup zones
fig1 = px.bar(
    df_query1,
    x='pickup_zone',
    y='trip_count',
    color='borough',
    title='Top 10 Busiest Pickup Zones',
    labels={'pickup_zone': 'Pickup Zone', 'trip_count': 'Number of Trips', 'borough': 'Borough'},
    text='trip_count'
)
fig1.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
fig1.update_layout(xaxis_tickangle=-45, height=500)
fig1.show()

print("\nInsight: The busiest pickup zones are concentrated in Manhattan, with areas like ")
print("Upper East Side, Midtown, and Penn Station seeing the highest taxi demand.")

## 3.3 Visualization 2: Average Fare by Hour (Line Chart)

In [ ]:
# Line chart: Average fare by hour
fig2 = px.line(
    df_query2,
    x='pickup_hour',
    y='avg_fare',
    title='Average Fare by Hour of Day',
    labels={'pickup_hour': 'Hour of Day', 'avg_fare': 'Average Fare ($)'},
    markers=True
)
fig2.update_layout(
    xaxis=dict(tickmode='linear', dtick=1),
    height=400
)
fig2.show()

print("\nInsight: Average fares peak during early morning hours (4-6 AM), likely due to ")
print("airport trips and longer distances traveled during off-peak hours.")

## 3.4 Visualization 3: Trip Distance Distribution (Histogram)

In [ ]:
# Histogram: Trip distance distribution
fig3 = px.histogram(
    df_features[df_features['trip_distance'] <= 30],  # Filter outliers
    x='trip_distance',
    nbins=50,
    title='Distribution of Trip Distances',
    labels={'trip_distance': 'Trip Distance (miles)', 'count': 'Number of Trips'}
)
fig3.update_layout(height=400)
fig3.show()

print("\nInsight: Most taxi trips are short-distance, with the majority under 5 miles.")
print("The right-skewed distribution shows typical urban taxi usage patterns.")

## 3.5 Visualization 4: Payment Type Distribution (Pie Chart)

In [ ]:
# Pie chart: Payment type distribution
fig4 = px.pie(
    df_query3,
    values='trip_count',
    names='payment_method',
    title='Payment Type Distribution',
    hole=0.4  # Donut chart
)
fig4.update_traces(textposition='inside', textinfo='percent+label')
fig4.update_layout(height=450)
fig4.show()

print("\nInsight: Credit cards are the dominant payment method, accounting for the vast ")
print("majority of transactions, reflecting modern payment preferences in NYC taxis.")

## 3.6 Visualization 5: Trips Heatmap by Day and Hour

In [ ]:
# Create heatmap data
day_order_map = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 
                 'Friday': 4, 'Saturday': 5, 'Sunday': 6}
df_features['day_num'] = df_features['pickup_day_of_week'].map(day_order_map)
heatmap_data = df_features.groupby(['day_num', 'pickup_hour']).size().reset_index(name='trips')
heatmap_pivot = heatmap_data.pivot(index='day_num', columns='pickup_hour', values='trips')

# Day labels
day_labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

fig5 = px.imshow(
    heatmap_pivot,
    labels=dict(x='Hour of Day', y='Day of Week', color='Trip Count'),
    x=list(range(24)),
    y=day_labels,
    title='Trip Volume Heatmap: Day of Week vs Hour',
    color_continuous_scale='YlOrRd',
    aspect='auto'
)
fig5.update_layout(height=400)
fig5.show()

print("\nInsight: Peak taxi usage occurs on weekday evenings (5-7 PM) and Friday/Saturday ")
print("nights. Early morning hours (2-5 AM) show the lowest demand across all days.")

## 3.7 Save Processed Data

In [ ]:
# Save processed data for the Streamlit app
processed_path = PROCESSED_DIR / "taxi_data_processed.parquet"
df_features.to_parquet(processed_path, index=False)
print(f"Processed data saved to: {processed_path}")
print(f"File size: {processed_path.stat().st_size / 1024 / 1024:.2f} MB")

---
# Summary
---

This notebook demonstrates a complete data pipeline for the NYC Yellow Taxi Trip dataset:

**Part 1 - Data Ingestion:**
- Downloaded ~3 million trip records programmatically
- Validated data structure and types
- Organized files in proper directory structure

**Part 2 - Data Transformation:**
- Cleaned data by removing nulls and invalid records
- Created derived features (duration, speed, temporal features)
- Performed SQL analysis using DuckDB

**Part 3 - Visualizations:**
- Prototyped 5 interactive visualizations using Plotly
- Calculated key business metrics
- Prepared data for Streamlit dashboard

The processed data is ready for deployment in the Streamlit dashboard (`app.py`).

In [ ]:
# Close DuckDB connection
con.close()
print("Analysis complete!")

---
# AI Tools Used
---

This assignment was completed with assistance from **GitHub Copilot** powered by Claude (Anthropic).

## How AI Was Used

### 1. Code Generation & Structure
- Generated boilerplate code for data ingestion pipeline
- Suggested function structures for data cleaning and validation
- Provided code patterns for DuckDB SQL integration

### 2. Data Transformation
- Assisted with Pandas operations for data cleaning
- Suggested approaches for feature engineering (trip duration, speed calculations)
- Helped structure temporal feature extraction

### 3. SQL Queries
- Generated DuckDB SQL query syntax
- Suggested aggregation patterns for business analytics
- Optimized query structure for performance

### 4. Visualizations
- Provided Plotly Express code patterns
- Suggested chart customizations and color schemes
- Helped structure the visualization layout

### 5. Documentation & Code Quality
- Generated docstrings and comments
- Suggested README structure and content
- Assisted with requirements.txt dependencies

## Verification
All AI-generated code was reviewed, tested, and modified as needed to ensure correctness and alignment with assignment requirements.